In [ ]:
import sys
import os
dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
from huggingface_hub import login

from utils.helpers import find_root_dir

import numpy as np
import seaborn as sns

In [ ]:
ARDT_DIR = find_root_dir()
VERSION_OF_COMBO = "v1"  # PLEASE CHANGE TO AVOID OVERRIDING!!!!!

In [ ]:
total_episodes = 1000
episode_len = 1000
total_steps = total_episodes * episode_len  # ideally

In [ ]:
dataset_names = ["arrl_nrmdp_train_halfcheetah_v4", "d4rl_expert_halfcheetah"]
weights = [0.5, 0.5]

In [ ]:
episodes_per_ds = [int(w * total_episodes) for w in weights]
super_dataset = None

for dataset_name, eps in zip(dataset_names, episodes_per_ds):
    # load dataset
    ds = load_from_disk(f"{ARDT_DIR}/datasets/{dataset_name}")
    # select last episodes_per_ds episodes
    len_ds = len(ds['observations'])
    ds = ds.select(range(len_ds-eps, len_ds))
    # visualise selection of dataset
    returns = (np.array(ds['rewards'])).sum(axis=1)
    sns.displot(returns, kind="kde", bw_adjust=0.5);
    # add to super dataset
    if super_dataset is None:
        super_dataset = ds
    else:
        ds = ds.cast(super_dataset.features)
        super_dataset = concatenate_datasets([super_dataset, ds])

In [ ]:
def compute_sum(ds):
    return {'returns': sum(ds['rewards'])}

ds_to_vis = super_dataset.map(compute_sum)
sns.displot(ds_to_vis['returns'], kind="kde", bw_adjust=0.5);

In [ ]:
super_dataset.save_to_disk(f'{ARDT_DIR}/datasets/dataset_combo_{VERSION_OF_COMBO}')